In [ ]:
import socket

def start_server(host='0.0.0.0', port=12345):
    # Create a TCP socket
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    # Bind to all interfaces on the chosen port
    server_socket.bind((host, port))
    server_socket.listen(1)
    print(f"Server listening on {host}:{port}")
    
    # Accept a connection
    client_socket, client_address = server_socket.accept()
    print(f"Connection accepted from {client_address}")
    
    # Send a greeting message to the client
    client_socket.sendall(b"Hello from server!")
    
    # Receive response from the client
    data = client_socket.recv(1024)
    print("Received from client:", data.decode())
    
    # Close the connection
    client_socket.close()
    server_socket.close()

if __name__ == '__main__':
    start_server()


In [ ]:
import socket

# A dictionary to store clients: {client_id: (ip, port)}
clients = {}

def start_server(host="0.0.0.0", port=12345):
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.bind((host, port))
    print(f"UDP Rendezvous server listening on {host}:{port}")
    
    while True:
        data, addr = sock.recvfrom(1024)
        message = data.decode().strip()
        print(f"Received from {addr}: {message}")
        tokens = message.split()
        if tokens[0].upper() == "REGISTER" and len(tokens) >= 2:
            client_id = tokens[1]
            clients[client_id] = addr
            sock.sendto(f"REGISTERED {client_id}".encode(), addr)
            print(f"Registered {client_id} at {addr}")
            # For demonstration: if two clients are registered, exchange info.
            if len(clients) >= 2:
                # This example assumes two specific IDs: "client1" and "client2"
                if "client1" in clients and "client2" in clients:
                    addr1 = clients["client1"]
                    addr2 = clients["client2"]
                    # Send each client the other's public endpoint.
                    sock.sendto(f"PEER {addr2[0]} {addr2[1]}".encode(), addr1)
                    sock.sendto(f"PEER {addr1[0]} {addr1[1]}".encode(), addr2)
                    print(f"Exchanged peer info between client1 and client2")
        else:
            sock.sendto(b"UNKNOWN COMMAND", addr)

if __name__ == '__main__':
    start_server()


In [ ]:
import socket

def run_server(bind_ip="0.0.0.0", bind_port=5005):
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.bind((bind_ip, bind_port))
    print(f"UDP server listening on {bind_ip}:{bind_port}")

    while True:
        data, addr = sock.recvfrom(1024)
        print("Received from", addr, ":", data.decode())
        # Reply back to confirm connectivity
        sock.sendto(b"ACK: " + data, addr)

if __name__ == "__main__":
    run_server()


In [1]:
import socket

known_port = 50002

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind(('0.0.0.0', 55555))

while True:
    clients = []

    while True:
        data, address = sock.recvfrom(128)

        print('connection from: {}'.format(address))
        clients.append(address)

        sock.sendto(b'ready', address)

        if len(clients) == 2:
            print('got 2 clients, sending details to each')
            break

    c1 = clients.pop()
    c1_addr, c1_port = c1
    c2 = clients.pop()
    c2_addr, c2_port = c2

    sock.sendto('{} {} {}'.format(c1_addr, c1_port, known_port).encode(), c2)
    sock.sendto('{} {} {}'.format(c2_addr, c2_port, known_port).encode(), c1)

: 

In [ ]:
import socket

def nat_server():
    server = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    server.bind(('0.0.0.0', 5000))
    print("Rendezvous server listening on port 5000...")
    
    clients = []
    
    while True:
        data, addr = server.recvfrom(1024)
        print(f"Received {data.decode()} from {addr}")
        if addr not in clients:
            clients.append(addr)
        
        # Once we have at least two clients, send each the other's address.
        if len(clients) >= 2:
            addr1, addr2 = clients[0], clients[1]
            msg1 = f"{addr2[0]}:{addr2[1]}"
            msg2 = f"{addr1[0]}:{addr1[1]}"
            server.sendto(msg1.encode(), addr1)
            server.sendto(msg2.encode(), addr2)
            print(f"Sent peer info: {addr1} <--> {addr2}")
            # Optionally, reset the client list for subsequent pairs.
            clients = []

if __name__ == "__main__":
    nat_server()


In [10]:
import socket
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind(("0.0.0.0", 5000))
print("Server listening on 0.0.0.0:5000")

Server listening on 0.0.0.0:5000
